In [1]:
import numpy as np
import scipy as sp
import scipy.io as io
from sklearn.model_selection import train_test_split
from os import path as path
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Conv2D, Dense, Flatten, Reshape, Input
from tensorflow.keras.utils import to_categorical
import time
from collections import Counter
import pickle
from itertools import zip_longest
import csv
import qkeras as qkeras
from qkeras import *
from qkeras.utils import model_save_quantized_weights
import os

Loading Data

In [2]:
with open("events.pickle", 'rb') as fp:
    events = pickle.load(fp)

with open("sorted_ch.pkl", 'rb') as fp:
    sorted_ch = pickle.load(fp)
    
channel_groups ={   
    1: {   
        'channels': list(range(64)),
        'geometry': {  
            0: [0, 400],
            1: [0, 775],
            2: [0, 425],
            3: [0, 750],
            4: [0, 725],
            5: [0, 450],
            6: [0, 700],
            7: [0, 475],
            8: [250, 275],
            9: [250, 100],
            10: [250, 250],
            11: [250, 125],
            12: [250, 225],
            13: [250, 150],
            14: [250, 525],
            15: [250, 175],
            16: [250, 0],
            17: [250, 375],
            18: [250, 25],
            19: [250, 350],
            20: [250, 325],
            21: [250, 50],
            22: [250, 300],
            23: [250, 75],
            24: [0, 675],
            25: [0, 500],
            26: [0, 650],
            27: [0, 525],
            28: [0, 625],
            29: [0, 550],
            30: [0, 125],
            31: [0, 575],
            32: [250, 650],
            33: [250, 550],
            34: [250, 625],
            35: [250, 575],
            36: [250, 775],
            37: [250, 400],
            38: [250, 750],
            39: [250, 425],
            40: [0, 325],
            41: [0, 50],
            42: [0, 100],
            43: [0, 275],
            44: [0, 75],
            45: [0, 300],
            46: [0, 600],
            47: [0, 200],
            48: [0, 250],
            49: [0, 150],
            50: [0, 225],
            51: [0, 175],
            52: [0, 375],
            53: [0, 0],
            54: [0, 350],
            55: [0, 25],
            56: [250, 725],
            57: [250, 450],
            58: [250, 500],
            59: [250, 675],
            60: [250, 475],
            61: [250, 700],
            62: [250, 200],
            63: [250, 600]},
        'graph': []
        }
    }
channel_cord = channel_groups[1]["geometry"]

In [17]:
# re-numbering the channel-numbers
events_dict = {}
ch_dict = {}
for i, k in enumerate(events):
    events_dict[i] = events[k]
    ch_dict[i] = k

In [18]:
print(events.keys())
print("shape = (# of data, input recording channels, snippet sample size)")
for i, k in enumerate(events_dict):
    print(k,events_dict[k].shape)

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13, 14, 15, 16, 23, 28, 44, 46, 48, 49, 50, 51, 52, 53, 54, 55, 56])
shape = (# of data, input recording channels, snippet sample size)
0 (300, 49, 180)
1 (300, 49, 180)
2 (297, 49, 180)
3 (300, 49, 180)
4 (300, 49, 180)
5 (300, 49, 180)
6 (300, 49, 180)
7 (300, 49, 180)
8 (209, 49, 180)
9 (300, 49, 180)
10 (300, 49, 180)
11 (300, 49, 180)
12 (300, 49, 180)
13 (300, 49, 180)
14 (300, 49, 180)
15 (212, 49, 180)
16 (300, 49, 180)
17 (300, 49, 180)
18 (234, 49, 180)
19 (300, 49, 180)
20 (231, 49, 180)
21 (300, 49, 180)
22 (300, 49, 180)
23 (300, 49, 180)
24 (300, 49, 180)
25 (300, 49, 180)
26 (300, 49, 180)


In [19]:
### Take middle 1 ms of the snippet ###
shorten_dict = dict()
for key in events_dict:
    shorten_dict[key] = events_dict[key][:,:,75:105]

Split Train, Test

In [20]:
### split snippet into sub-snippets of size filter_width ###
def convert2D(data, filter_width):
    org_shape = data.shape
    X_2d = np.reshape(data, (org_shape[0], org_shape[1], -1, filter_width))
    X_2d = np.moveaxis(X_2d, [2,3],[3,2])
    return X_2d

In [21]:
### Define model ###
def create_model_2d(all_classes, filter_width, inp_shape):
    # inputshape = (49, 6, 15)
    model = Sequential()
    filter_cout = 2
    model.add(Conv2D(filters=filter_cout, kernel_size=(4,4), strides = (1,2), activation='relu', input_shape=inp_shape))
    model.add(Reshape((-1, filter_cout)))
    model.add(Conv1D(filters=2, kernel_size=4, strides = 2, activation='relu'))
    model.add(Flatten())
    model.add(Dense(all_classes, activation='softmax'))

    return model

In [22]:
### unifromly distribute units into test, train, validation ###
filter_width = 6

np.random.seed(2021)
x_data = []
y_data = []
x_train_reshaped = []
x_test_reshaped = []
x_val_reshaped = []
y_train = []
y_test = []
y_val = []

train_ratio = 0.75
validation_ratio = 0.1
test_ratio = 0.15

# events_dict_avg
events_dict_reDetect = shorten_dict
for unit in events_dict_reDetect:
    unit_waveform = []
    unit_y = []
    for waveforms in events_dict_reDetect[unit]:
        unit_waveform.append(waveforms)
        unit_y.append(unit)
        
    unit_waveform = np.array(unit_waveform)
    unit_y = np.array(unit_y)

    # train is now 75% of the entire data set
    x_train_N, x_test_N, y_train_N, y_test_N = train_test_split(unit_waveform, unit_y, test_size=1 - train_ratio, random_state=2021)

    # test is now 10% of the initial data set
    # validation is now 15% of the initial data set
    x_val_N, x_test_N, y_val_N, y_test_N = train_test_split(x_test_N, y_test_N, test_size=test_ratio/(test_ratio + validation_ratio), random_state=2021)
    

    y_train.append(y_train_N)
    y_test.append(y_test_N)
    y_val.append(y_val_N)

    x_train_reshaped.append(np.moveaxis(x_train_N, [1,2], [2,1]))
    x_test_reshaped.append(np.moveaxis(x_test_N, [1,2], [2,1]))
    x_val_reshaped.append(np.moveaxis(x_val_N, [1,2], [2,1]))

y_train = np.concatenate(y_train, axis=0)
y_test = np.concatenate(y_test, axis=0)
y_val = np.concatenate(y_val, axis=0)
y_train = y_train.reshape((y_train.shape[0],1))
y_test = y_test.reshape((y_test.shape[0],1))
y_val = y_val.reshape((y_val.shape[0],1))

x_train_reshaped = np.concatenate(x_train_reshaped, axis=0)
x_test_reshaped = np.concatenate(x_test_reshaped, axis=0)
x_val_reshaped = np.concatenate(x_val_reshaped, axis=0)

x_train_reshaped = np.moveaxis(x_train_reshaped, [1,2], [2,1])
x_test_reshaped = np.moveaxis(x_test_reshaped, [1,2], [2,1])
x_val_reshaped = np.moveaxis(x_val_reshaped, [1,2], [2,1])

x_train_reshaped = x_train_reshaped[:,sorted_ch,:]
x_test_reshaped = x_test_reshaped[:,sorted_ch,:]
x_val_reshaped = x_val_reshaped[:,sorted_ch,:]

x_train_reshaped = convert2D(x_train_reshaped, filter_width)
x_test_reshaped = convert2D(x_test_reshaped, filter_width)
x_val_reshaped = convert2D(x_val_reshaped, filter_width)


print(x_train_reshaped.shape, y_train.shape, x_val_reshaped.shape, y_val.shape, x_test_reshaped.shape, y_test.shape)
print(np.unique(y_train))

all_classes = len(np.unique(y_train))

(5835, 49, 6, 5) (5835, 1) (778, 49, 6, 5) (778, 1) (1170, 49, 6, 5) (1170, 1)
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26]


In [ ]:
### train and test models multiple trials ###
perGroup_list = []
headers = ['Partition', 'trail', 'test_acc', 'val_acc', 'val_loss', 'train_loss']
max_test_acc = 0
num_trials = 10
for i in range(num_trials):
    print(i)
    perGroup_dict = dict()
    perGroup_dict['Partition'] = i
    model = create_model_2d(all_classes, filter_width, x_train_reshaped[0].shape)
    model.summary()
    model.compile(
    loss=keras.losses.categorical_crossentropy,
                        optimizer=keras.optimizers.Adam(lr=0.0005), ## (0.001) originally
                        metrics=['accuracy'],
    )

    model_file_name = './models/CNN-s_explore_trial' + str(i) + '.wts.h5'

    # Train the model.
    t0 = time.time()
    history = model.fit(
        x_train_reshaped,
        to_categorical(y_train),
        epochs=600,
        validation_data=(x_val_reshaped, to_categorical(y_val)),
        verbose=1,
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100),
            keras.callbacks.ModelCheckpoint(model_file_name, monitor='val_loss', verbose=0, save_best_only=True, mode='auto')]
        )
    t1 = time.time()
    duration_ms = (t1-t0)*1000

    history_data = {'epoch':history.epoch, 'history':history.history}
    with open('./models/history_' + str(i) + '.pickle', 'wb') as outp:
        pickle.dump(history_data, outp, pickle.HIGHEST_PROTOCOL)

    model.load_weights(model_file_name)
    
    predicted_unit = model.predict(x_test_reshaped) # FINAL FUNCTION CALL
    
    print("Duration (ms):", duration_ms)

    val_acc = history.history['val_accuracy']
    max_acc, idx  = max([(val, i) for i, val in enumerate(val_acc)])
    val_loss = history.history['val_loss'][idx]
    train_acc = history.history['accuracy'][idx]
    train_loss = history.history['loss'][idx]

    acc = model.evaluate(x_test_reshaped, to_categorical(y_test))[1]
   
    best_model_file = './acc_finals/CNN_6final_best.wts'
    if max_test_acc < acc:
        model.save_weights(best_model_file)
        max_test_acc = acc
        print(max_test_acc)

    perGroup_dict['test_acc'] = acc
    perGroup_dict['val_acc'] = max_acc
    perGroup_dict['val_loss'] = val_loss
    perGroup_dict['train_loss'] = train_loss

    perGroup_list.append(perGroup_dict)
perPartition_file = './acc/perPartition_acc.csv'
with open(perPartition_file, "w") as csvFile:
    writer = csv.DictWriter(csvFile, fieldnames=headers)
    writer.writeheader()
    writer.writerows(perGroup_list)

In [507]:
model= create_model_2d(all_classes, filter_width, x_train_reshaped[0].shape)
model.summary()
model.compile(
loss=keras.losses.categorical_crossentropy,
                    optimizer=keras.optimizers.Adam(lr=0.0005), ## (0.001) originally
                    metrics=['accuracy'],
)

model.load_weights(best_model_file)
acc = model.evaluate(x_test_reshaped, to_categorical(y_test))[1]
print(acc)

Model: "sequential_236"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_236 (Conv2D)          (None, 46, 2, 2)          162       
_________________________________________________________________
reshape_235 (Reshape)        (None, 92, 2)             0         
_________________________________________________________________
conv1d_235 (Conv1D)          (None, 45, 2)             18        
_________________________________________________________________
flatten_235 (Flatten)        (None, 90)                0         
_________________________________________________________________
dense_235 (Dense)            (None, 27)                2457      
Total params: 2,637
Trainable params: 2,637
Non-trainable params: 0
_________________________________________________________________
37/37 [==============================] - 0s 1ms/step - loss: 0.2691 - accuracy: 0.9308
0.9307692050933838


In [ ]:
### quantized model ###

def CreateQModel(all_classes, inp_shape, bit_conv, bit_dense):
    x = x_in = Input(inp_shape)
    x = QConv2D(2, (4,4), strides = (1,2), 
    kernel_quantizer = "quantized_bits(" + str(bit_conv) + ")",
    bias_quantizer = "quantized_bits(" + str(bit_conv) + ")",
    name = "conv1d_1") (x)
    x = QActivation("quantized_relu(" + str(bit_conv) + ")", name="act_1")(x)
    x = Reshape((-1, 2))(x)

    x = QConv1D(2, 4, strides = 2, 
    kernel_quantizer = "quantized_bits(" + str(bit_conv) + ")",
    bias_quantizer = "quantized_bits(" + str(bit_conv) + ")",
    name = "conv1d_2") (x)
    x = QActivation("quantized_relu(" + str(bit_conv) + ")", name="act_2")(x)

    x = Flatten(name="flatten")(x)
    x = QDense(all_classes,
        kernel_quantizer="quantized_bits(" + str(bit_dense) + ")",
        bias_quantizer="quantized_bits(" + str(bit_dense) + ")",
        name="dense")(x)
    x = Activation("softmax", name="softmax")(x)
    
    model = Model(inputs=x_in, outputs=x)
    
    return model

In [ ]:
def modelState(qmodel):
    model_ops = extract_model_operations(qmodel)
    model_table = dict()
    ops_table = defaultdict(lambda: 0)

    for name in sorted(model_ops):
        mode, _, sizes, signs = model_ops[name]["type"]
        number = model_ops[name]["number_of_operations"]
        sign = "s" if sum(signs) > 0 else "u"
        op_name = sign + mode + "_" + str(sizes[0]) + "_" + str(sizes[1])
        ops_table[op_name] += number
        model_table[str(name)] = (number, op_name)
    
    weight_table = dict()

    for name in sorted(model_ops):
        weight_type = model_ops[name]["type_of_weights"]
        n_weights = model_ops[name]["number_of_weights"]
        if isinstance(weight_type, list):
            for i, (w_type, w_number) in enumerate(zip(weight_type, n_weights)):
                _, w_sizes, _ = w_type
                weight_table[str(name) + "_weights"] = (w_number, w_sizes)
        else:
            _, w_sizes, _ = weight_type
            weight_table[str(name) + "_weights"] = (n_weights, w_sizes)
            _, b_sizes, _ = model_ops[name]["type_of_bias"]
            b_number = model_ops[name]["number_of_bias"]
            weight_table[str(name) + "_bias"] = (b_number, b_sizes)
    new_ops = {key:val for key, val in ops_table.items()}
    return model_table, new_ops, weight_table

In [ ]:
perGroup_list = []
headers = ['conv_bit', 'dense_bit', 'trial', 'test_acc', 'val_acc', 'val_loss', 'train_loss']
conv_bit_list = [8]
dense_bit_list = [4]
for conv_bit in conv_bit_list:
    for dense_bit in dense_bit_list:
        if conv_bit >= dense_bit:
            for i in range(1):
                print(conv_bit, dense_bit)
                perGroup_dict = dict()
                perGroup_dict['trial'] = i
                perGroup_dict['conv_bit'] = conv_bit
                perGroup_dict['dense_bit'] = dense_bit
                model = CreateQModel(all_classes, x_train_reshaped[0].shape, conv_bit, dense_bit)
                ops_table  = modelState(model)

                model.compile(
                loss=keras.losses.categorical_crossentropy,
                                    optimizer=keras.optimizers.Adam(lr=0.0005), ## (0.001) originally
                                    metrics=['accuracy'],
                )

                model_file_name = './qkeras/model/' + str(conv_bit) + '-' + str(dense_bit) + 'qkeras_t' + str(i) + '.wts.h5'

                t0 = time.time()
                # Train the model.
                history = model.fit(
                x_train_reshaped,
                to_categorical(y_train),
                epochs=300,
                validation_data=(x_val_reshaped, to_categorical(y_val)),
                verbose=1,
                callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=150),
                    keras.callbacks.ModelCheckpoint(model_file_name, monitor='val_accuracy', verbose=0, save_best_only=True, mode='auto')]
                )
                t1 = time.time()

                history_data = {'epoch':history.epoch, 'history':history.history}

                model.load_weights(model_file_name)

                predicted_unit = model.predict(x_test_reshaped) # FINAL FUNCTION CALL
                
                duration_ms = (t1-t0)*1000
                print("Duration (ms):", duration_ms)

                val_acc = history.history['val_accuracy']
                max_acc, idx  = max([(val, i) for i, val in enumerate(val_acc)])
                val_loss = history.history['val_loss'][idx]
                train_acc = history.history['accuracy'][idx]
                train_loss = history.history['loss'][idx]



                acc = model.evaluate(x_test_reshaped, to_categorical(y_test))[1]
                perGroup_dict['test_acc'] = acc
                perGroup_dict['val_acc'] = max_acc
                perGroup_dict['val_loss'] = val_loss
                perGroup_dict['train_loss'] = train_loss

                perGroup_list.append(perGroup_dict)

perPartition_file = './qkeras/acc/qkeras_quantized_input_test.csv'

with open(perPartition_file, "w") as csvFile:
    writer = csv.DictWriter(csvFile, fieldnames=headers)
    writer.writeheader()
    writer.writerows(perGroup_list)